In [ ]:
! pip install mtcnn

     |████████████████████████████████| 2.3MB 2.8MB/s 
  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp36-none-any.whl size=2257690 sha256=4ab07244fd6ddf1e01eb537b734e84548eb553b25ef2176799e0f259268c856c
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
Successfully built mtcnn


In [ ]:
from sklearn.externals import joblib
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os import mkdir
from os.path import isdir
from PIL import Image
import cv2
from mtcnn.mtcnn import MTCNN

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
import numpy as np
# from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from keras.models import load_model
from random import choice
# from numpy import expand_dims
from google.colab.patches import cv2_imshow

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data = np.load('/content/drive/My Drive/twice_npy/twice-members-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
del data
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)

In [ ]:
filename = '/content/drive/My Drive/twice_npy/twice_members.sav'
loaded_model = joblib.load(filename)
# result = loaded_model.predict(testX)

In [ ]:
# name_list = out_encoder.classes_
# res_names = []
# for i in result :
#   res_names.append(name_list[i])

In [ ]:
# sum(result == testy)


In [ ]:
# len(testy)

In [ ]:
# ! wget https://www.wallpapermaiden.com/wallpaper/27784/download/2560x1440/twice-south-korean-girls.jpeg

In [ ]:
model = load_model('/content/drive/My Drive/keras-facenet/model/facenet_keras.h5')









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
def get_embedding(model, face_pixels):
	# scale pixel values
	face_pixels = face_pixels.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	samples = np.expand_dims(face_pixels, axis=0)
	# make prediction to get embedding
	yhat = model.predict(samples)
	return yhat[0]

In [ ]:
detector = MTCNN()
# filename = "twice-south-korean-girls.jpeg"
# image = cv2.imread("twice-south-korean-girls.jpeg")
# result = detector.detect_faces(image)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.



In [ ]:
# image = Image.open(filename)
# image = image.convert('RGB')
# pixels = np.asarray(image)
# result = detector.detect_faces(pixels)
# required_size=(160, 160)
# font = cv2.FONT_HERSHEY_SIMPLEX
# # len(result)


In [ ]:
def detect_faces(pixels,detector) :
#   image = Image.open(filename)
#   image = image.convert('RGB')
#   pixels = np.asarray(image)
  result = detector.detect_faces(pixels)
  required_size=(160, 160)
  font = cv2.FONT_HERSHEY_SIMPLEX
  # len(result)
  for i in range(len(result)) :
    bounding_box = result[i]['box']
    keypoints = result[i]['keypoints']
    x1, y1, width, height = bounding_box
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
  #   print(face_array.shape)
    embedding = get_embedding(model, face_array)
    l = np.asarray([embedding])
    X_test = in_encoder.transform(l)
    yhat_class = loaded_model.predict(X_test)
    yhat_prob = loaded_model.predict_proba(X_test)
    # get name
    class_index = yhat_class[0]
    
    class_probability = yhat_prob[0,class_index] * 100
    predict_names = out_encoder.inverse_transform(yhat_class)
#     print(predict_names)
    if class_probability > 78 :
      s = predict_names[0].split("_")[-1] 
      cv2.putText(pixels, s, (x1,y1-20), font, 1, (0, 255, 0), 2)
  return pixels
  #   cv2.rectangle(image,
  #               (bounding_box[0], bounding_box[1]),
  #               (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
  #               (0,155,255),
  #               2)
  # cv2.imwrite("twice_drawn.jpg", image)
  # plt.imsave(name + filename,face)

In [ ]:
# cv2.imwrite("twice_drawn3.jpg", pixels)
# plt.imsave("twice_drawn5.jpg", pixels)

In [ ]:
import cv2
import numpy as np
 
# Create a VideoCapture object
cap = cv2.VideoCapture('/content/drive/My Drive/twice_frames/twice_signal.mp4')
 
# Check if camera opened successfully
if (cap.isOpened() == False): 
  print("Unable to read camera feed")
 
# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
 
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter('signal.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 24, (frame_width,frame_height))
i = 0
while(True):
  ret, frame = cap.read()
  if ret == True: 
    if i%4 == 0 :
      result = detector.detect_faces(frame)
      pix = frame
    
    required_size=(160, 160)
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(len(result)) :
      bounding_box = result[i]['box']
      keypoints = result[i]['keypoints']
      x1, y1, width, height = bounding_box
      x1, y1 = abs(x1), abs(y1)
      x2, y2 = x1 + width, y1 + height
      face = pix[y1:y2, x1:x2]
      image = Image.fromarray(face)
      image = image.resize(required_size)
      face_array = np.asarray(image)
      embedding = get_embedding(model, face_array)
      l = np.asarray([embedding])
      X_test = in_encoder.transform(l)
      yhat_class = loaded_model.predict(X_test)
      yhat_prob = loaded_model.predict_proba(X_test)
      class_index = yhat_class[0]
      class_probability = yhat_prob[0,class_index] * 100
      predict_names = out_encoder.inverse_transform(yhat_class)
      if class_probability > 60 :
        s = predict_names[0].split("_")[-1] 
        cv2.putText(frame, s, (x1,y1-20), font, 1, (0, 255, 0), 2)
    # Write the frame into the file 'output.avi'
    out.write(frame)
    i += 1
    # Display the resulting frame    
#     cv2.imshow('frame',frame)
 
    # Press Q on keyboard to stop recording
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else:
    break 
 
# When everything done, release the video capture and video write objects
cap.release()
out.release()
 
# Closes all the frames
cv2.destroyAllWindows()

KeyboardInterrupt: ignored

In [ ]:
! zip -r -qq signal_1.zip signal.avi

In [ ]:
# !pip install -U -q PyDrive
# !git clone https://github.com/Joshua1989/python_scientific_computing.git
!git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
!mv /tmp/colab_util_repo/colab_util.py colab_util.py 
!rm -r /tmp/colab_util_repo

Cloning into '/tmp/colab_util_repo'...
remote: Enumerating objects: 40, done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 40
Unpacking objects: 100% (40/40), done.


In [ ]:
from colab_util import *
drive_handler = GoogleDriveHandler()

In [ ]:
test_folder_id = drive_handler.create_folder('twice_frames')
test_folder_id

twice_frames already exists


'1qVdk1arrS9ai5VK1hjA9FciEZE9PIXAz'

In [ ]:
drive_handler.upload("signal_1.zip", parent_path='twice_frames')
drive_handler.list_folder(test_folder_id, max_depth=1)

[{'id': '151OMRrIRXTddGijCf_nt-Qw3sqazOc2l',
  'link': 'https://drive.google.com/file/d/151OMRrIRXTddGijCf_nt-Qw3sqazOc2l/view?usp=drivesdk',
  'mimeType': 'application/zip',
  'title': 'signal_1.zip'},
 {'id': '1MMbYIrG5l5Zl_ciAs72F4sdhDf-afPdN',
  'link': 'https://drive.google.com/file/d/1MMbYIrG5l5Zl_ciAs72F4sdhDf-afPdN/view?usp=drivesdk',
  'mimeType': 'application/zip',
  'title': 'cheer_up.zip'},
 {'id': '1dKRKOLoaQJIYzfOPA2paJcJTsWG4SAcF',
  'link': 'https://drive.google.com/file/d/1dKRKOLoaQJIYzfOPA2paJcJTsWG4SAcF/view?usp=drivesdk',
  'mimeType': 'application/zip',
  'title': 'likey.zip'},
 {'id': '16teUghZlEACTDGFIbbFl4JlPv78H0gdY',
  'link': 'https://drive.google.com/file/d/16teUghZlEACTDGFIbbFl4JlPv78H0gdY/view?usp=drivesdk',
  'mimeType': 'application/zip',
  'title': 'fancy.zip'},
 {'id': '1-MuL8e8DdHf1PiX7KBGz2jTe9QulhPH_',
  'link': 'https://drive.google.com/file/d/1-MuL8e8DdHf1PiX7KBGz2jTe9QulhPH_/view?usp=drivesdk',
  'mimeType': 'application/zip',
  'title': 'tt.zip'

In [ ]:
! sudo apt install caffe-cpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  caffe-tools-cpu cython3 fonts-lyx javascript-common libblosc1 libcaffe-cpu1
  libgflags2.2 libgoogle-glog0v5 libjs-jquery libjs-jquery-ui libleveldb1v5
  liblmdb0 python-matplotlib-data python-tables-data python3-bs4
  python3-caffe-cpu python3-chardet python3-cycler python3-dateutil
  python3-decorator python3-gflags python3-h5py python3-html5lib
  python3-ipython python3-ipython-genutils python3-leveldb python3-lxml
  python3-matplotlib python3-networkx python3-nose python3-numexpr
  python3-olefile python3-pandas python3-pandas-lib python3-pexpect
  python3-pickleshare python3-pil python3-pkg-resources python3-prompt-toolkit
  python3-protobuf python3-ptyprocess python3-pygments python3-pyparsing
  python3-pywt python3-scipy python3-simplegeneric python3-six python3-skimage
  python3-skimage-lib python3-tables python3-tables-lib pytho